# Intro to Microsoft Agent Framework (MAF)

Learn the patterns used in the main app (`app/main.py`):
1. **Foundry Agent** - Azure AI Agent Service (RealtimeAssistant)
2. **Fabric Data Agent** - OneLake/SQL queries (AirlineOpsContext)
3. **Orchestration** - Router + `.as_tool()` pattern

**Prerequisites:** `az login`, env vars configured

In [ ]:
# Install dependencies
!pip install -q agent-framework azure-identity azure-ai-projects openai>=1.70.0

In [ ]:
import os
import asyncio
from typing import Annotated
from pydantic import Field

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Check env vars
required = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_OPENAI_API_ENDPOINT", "AZURE_OPENAI_KEY"]
missing = [k for k in required if not os.getenv(k)]
print("✅ Ready" if not missing else f"⚠️ Missing: {missing}")

## 1. Foundry Agent (RealtimeAssistant)
Connect to Azure AI Agent Service - used for Bing search, weather, file search in `app/main.py`.

In [ ]:
async def build_foundry_agent(agent_id: str) -> ChatAgent:
    """Connect to existing Azure AI Foundry agent."""
    endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]
    cred = AzureCliCredential()
    project = AIProjectClient(endpoint=endpoint, credential=cred)
    
    chat_client = AzureAIAgentClient(project_client=project, agent_id=agent_id, credential=cred)
    return ChatAgent(
        chat_client=chat_client,
        name="FoundryAgent",
        description="Azure AI Foundry agent",
    )

# Replace with your agent ID
AGENT_ID = "asst_c5nsKgXECwvlBafUNpOFjuZO"
foundry_agent = await build_foundry_agent(AGENT_ID)

result = await foundry_agent.run("What's the weather in Madrid?")
print(result.text if hasattr(result, 'text') else result)

## 2. Fabric Data Agent (AirlineOpsContext)
Query OneLake data via Fabric IQ - used for airline operations queries in `app/main.py`.

In [ ]:
import time, uuid
from azure.identity import InteractiveBrowserCredential
from openai import OpenAI
from openai._models import FinalRequestOptions

FABRIC_ENDPOINT = "https://msitapi.fabric.microsoft.com/v1/workspaces/00ae18cb-e789-4d42-be8d-a5b47e524e22/aiskills/1e363010-6005-48da-b370-7906177a760e/aiassistant/openai"
_cred = InteractiveBrowserCredential()

class FabricOpenAI(OpenAI):
    def __init__(self, base_url: str, **kwargs):
        super().__init__(api_key="", base_url=base_url, default_query={"api-version": "2024-05-01-preview"}, **kwargs)
    
    def _prepare_options(self, options: FinalRequestOptions) -> None:
        options.headers = {**getattr(options, 'headers', {}),
            "Authorization": f"Bearer {_cred.get_token('https://api.fabric.microsoft.com/.default').token}",
            "ActivityId": str(uuid.uuid4())}
        return super()._prepare_options(options)

fabric_client = FabricOpenAI(base_url=FABRIC_ENDPOINT)

def ask_fabric(question: str, timeout_sec: int = 300) -> str:
    """Query Fabric Data Agent."""
    assistant = fabric_client.beta.assistants.create(model="not-used")
    thread = fabric_client.beta.threads.create()
    try:
        fabric_client.beta.threads.messages.create(thread_id=thread.id, role="user", content=question)
        run = fabric_client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
        
        start = time.time()
        while run.status not in {"completed", "failed", "cancelled"}:
            if time.time() - start > timeout_sec: raise TimeoutError("Timeout")
            time.sleep(2)
            run = fabric_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        
        if run.status != "completed": return f"[Run ended: {run.status}]"
        msgs = fabric_client.beta.threads.messages.list(thread_id=thread.id, order="asc")
        return "\n".join(c.text.value for m in msgs.data if m.role == "assistant" for c in m.content if c.type == "text")
    finally:
        try: fabric_client.beta.threads.delete(thread_id=thread.id)
        except: pass

print(ask_fabric("What data do you have access to?"))

## 3. Wrap Fabric as a Tool
Expose Fabric queries as a callable tool for an orchestrator agent.

In [ ]:
def fabric_query(question: Annotated[str, Field(description="Data query")]) -> str:
    """Query operational data from Fabric."""
    return ask_fabric(question)

unified_agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        deployment_name=os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID"),
    ),
    name="UnifiedAgent",
    description="Agent with Fabric data access",
    instructions="Call fabric_query for data questions. Answer general questions directly.",
    tools=[fabric_query],
)

result = await unified_agent.run("What's the product A MARD comparison with Product B?")
print(result)

## 4. Router Pattern (Deterministic)
Simple keyword-based routing - choose between Foundry (realtime) and Fabric (data).

In [ ]:
def route(query: str) -> str:
    q = query.lower()
    if any(w in q for w in ["flights", "baggage", "airport", "routes", "sla"]):
        return "fabric"  # AirlineOpsContext
    return "foundry"  # RealtimeAssistant (weather, time, web search)

async def ask(query: str) -> str:
    choice = route(query)
    print(f"🎯 → {choice}")
    if choice == "fabric": 
        return (await unified_agent.run(query)).text
    return (await foundry_agent.run(query)).text

# Test
print(await ask("What flights are delayed at ORD?"))

## 5. Router Pattern with .as_tool()
This is how `app/main.py` works - LLM orchestrator decides which sub-agent to call.

In [ ]:
# This mirrors the AirlineIntelligentAssistant orchestrator in app/main.py
router_agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        deployment_name=os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID"),
    ),
    name="Orchestrator",
    instructions=(
        "Route to the appropriate agent:\n"
        "• AirlineOpsContext → flights, baggage, routes, airports, SLA metrics\n"
        "• RealtimeAssistant → weather, time, web search, current events"
    ),
    tools=[
        unified_agent.as_tool(name="AirlineOpsContext", description="Query Fabric for airline operations data"),
        foundry_agent.as_tool(name="RealtimeAssistant", description="Weather, time, web search"),
    ],
)

result = await router_agent.run("What flights are delayed and what's the weather in Chicago?")
print(result)

## 6. Magentic Orchestration (Multi-round)
Advanced orchestration with streaming - useful for complex multi-step tasks.

In [ ]:
from agent_framework import MagenticBuilder, MagenticFinalResultEvent, MagenticAgentDeltaEvent

async def run_magentic(query: str):
    workflow = MagenticBuilder().participants(router=router_agent).with_standard_manager(max_round_count=6).build()
    
    async for event in workflow.run_stream(query):
        if isinstance(event, MagenticAgentDeltaEvent) and event.text:
            print(event.text, end="", flush=True)
        elif isinstance(event, MagenticFinalResultEvent) and event.message:
            print(f"\n\n=== FINAL ===\n{event.message.text}")

await run_magentic("Pull Q3 revenue KPIs and link me to the source file.")

## 7. Observability (Middleware)
Add logging to see which tools get called - helpful for debugging.

In [ ]:
from agent_framework import FunctionInvocationContext, ChatContext

async def log_tools(ctx: FunctionInvocationContext, nxt):
    print(f"[Tool] {ctx.function.name}({ctx.arguments})")
    await nxt(ctx)

async def log_llm(ctx: ChatContext, nxt):
    print(f"[LLM] {len(ctx.messages)} messages")
    await nxt(ctx)

router_agent.function_middlewares = [log_tools]
router_agent.chat_middlewares = [log_llm]
print("✅ Middleware attached")